In [218]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

from p1_graphs import *
import altair as alt
data = get_accident_data(fname="dataset_v1.csv",sample=True)
accident_data = get_weather_data(data,fname = "weather.csv")

data.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y
c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\IPython\core\interactiveshell.py:3526: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index_left,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date,weekday,covid,BoroName,index,BoroCode,Shape_Leng,Shape_Area
0,96668,2018-07-02T00:00:00Z,6:30,BRONX,40.812126,-73.904190,Bike,2018-07-02,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,61404,2018-08-20T00:00:00Z,12:59,BRONX,40.848114,-73.827690,Sedan,2018-08-20,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,68315,2018-08-04T00:00:00Z,20:04,NaN,40.875195,-73.888565,Sedan,2018-08-04,weekend,before,Bronx,4,2,464392.991824,1.186925e+09
0,84481,2018-07-27T00:00:00Z,8:20,BRONX,40.858604,-73.885450,Sedan,2018-07-27,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,97519,2018-06-27T00:00:00Z,1:37,BRONX,40.879215,-73.858475,Sedan,2018-06-27,weekday,before,Bronx,4,2,464392.991824,1.186925e+09


In [219]:
mapa = get_map()
ny_df, bur = get_buroughs(mapa)
# normalize bur and accident data BoroName
bur = bur.reset_index()
accident_data.head()
# create properties.name column equal to BoroName
accident_data['properties.name'] = accident_data['BoroName']


accident_data['weekday'] = accident_data['date'].dt.weekday
# make weekend column
accident_data['weekend'] = accident_data['weekday'].apply(lambda x: 1 if x > 4 else 0)

# make column with week number
accident_data['week'] = accident_data['date'].dt.week
# month column
accident_data['month'] = accident_data['date'].dt.month

# for each month get the minimum week number
min_week = accident_data.groupby(['month'])['week'].min().reset_index()
# merge with accident data
accident_data = pd.merge(accident_data,min_week,on='month',how='left')
accident_data['week'] = accident_data['week_x'] - accident_data['week_y'] + 1


c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\pamar\Desktop\Q5\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y
C:\Users\pamar\AppData\Local\Temp\ipykernel_6348\2291498277.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  accident_data['week'] = accident_data['date'].dt.week


In [220]:

colors = {"bg": "#eff0f3", "col1": "#d8b365", "col2": "#5ab4ac"}
w = 600
h = 400
ratio = 0.2
# accident_data = get_weather_data(data,fname = "weather.csv")
print(accident_data.columns)
selection_cond = alt.selection_multi(on="click", empty="all", fields=["conditions"])
selection_buro = alt.selection_point(fields=["properties.name"], empty="all")
month_dropdown = alt.binding_select(options=[[6,7,8],6,7,8])
selection_month = alt.selection_point(fields=['month'], bind=month_dropdown)

weekday_dropdown = alt.binding_select(options=[[0,1,2,3,4],[5,6]],name='weekday',labels=['weekday','weekend'])
selection_weekday = alt.selection_point(fields=['weekday'], bind=weekday_dropdown)

selection_day = alt.selection_point(fields=['weekday','week','month'])
# choropleth, _ = plot_map(hex_data, mapa, ny_df, bur,selection_buro,selection_cond)


ny = "https://raw.githubusercontent.com/pauamargant/VI_P1/main/resources/new-york-city-boroughs.geojson"
data_geojson_remote = alt.Data(
    url=ny, format=alt.DataFormat(property="features", type="json")
)
base = (
    alt.Chart(data_geojson_remote)
    .mark_geoshape()
    .properties(
        width=500,
        height=300,
    )
    .project(type="albersUsa")
    .add_params(selection_buro)
    .encode(opacity=alt.condition(selection_buro, alt.value(0.8), alt.value(0.2)))
)
points = (
    alt.Chart(accident_data)
    .transform_filter(selection_month & selection_weekday)
    .mark_circle()
    .encode(
        longitude="LONGITUDE:Q",
        latitude="LATITUDE:Q",
        size=alt.value(2),
        color=alt.value("red"),
        opacity=alt.condition(selection_buro, alt.value(1), alt.value(0)),
    )
    .add_params(selection_buro)
)

bar_chart = (
    alt.Chart(accident_data)
    .mark_bar()
    .transform_filter(selection_cond  & selection_month & selection_weekday)
    .encode(
        x=alt.X("count()"),
        y=alt.Y("properties\\.name"),
        opacity=alt.condition(selection_buro, alt.value(1), alt.value(0.4)),
    )
)
geo_view = (base + points) | bar_chart


bars = (
    alt.Chart(accident_data)
    .transform_filter(selection_buro  & selection_month & selection_weekday)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .mark_bar(height=3, orient="horizontal")
    .encode(
        y=alt.Y("conditions:N").sort("x"),
        x="min(diff):Q",
        color=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        fill=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * (1 - ratio), height=h)
    .add_params(selection_cond)
)
dots = (
    alt.Chart(accident_data)
    .transform_filter(selection_buro  & selection_month & selection_weekday)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .mark_point(height=3, orient="horizontal", size=100)
    .encode(
        y=alt.Y("conditions:N").sort("x"),
        x="min(diff):Q",
        color=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        fill=alt.condition(
            alt.datum.diff > 0,
            alt.value(colors["col2"]),  # The positive color
            alt.value(colors["col1"]),  # The negative color
        ),
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * (1 - ratio), height=h)
    .add_params(selection_cond)
)
bar_legend = (
    alt.Chart(accident_data)
    .mark_rect()
    .transform_filter(selection_buro  & selection_month & selection_weekday)
    .transform_joinaggregate(day_count="count()", groupby=["date"])
    .transform_joinaggregate(per_day_cond="mean(day_count)", groupby=["conditions"])
    .transform_joinaggregate(per_day_mean="mean(day_count)", groupby=[])
    .transform_calculate(diff="datum.per_day_cond - datum.per_day_mean")
    .encode(
        y=alt.Y("conditions:N", sort=alt.EncodingSortField(field="diff")),
        color="count()",
        opacity=alt.condition(selection_cond, alt.value(1), alt.value(0.2)),
    )
    .properties(width=w * ratio, height=h)
    .add_params(selection_cond)
)


# subtract minimum week number from week number

# get number of week in month

# make column with month number

calendar = alt.Chart(accident_data).mark_rect().transform_filter(selection_cond & selection_buro).encode(
    x = alt.X('weekday:O'),
    y = alt.Y('week:O',scale=alt.Scale(domain=[5,4,3,2,1])),
    row = alt.Column('month'),
    color = alt.Color('count()'),
    opacity=alt.condition(selection_month & selection_weekday, alt.value(1), alt.value(0.2))
).add_params(selection_month, selection_weekday)

# .configure_scale(
#     bandPaddingInner=.1).add_params(date_selector,month_selection, weekday_selection)

geo_view & (bars + dots | bar_legend | calendar) 

Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE',
       'LONGITUDE', 'VEHICLE TYPE CODE 1', 'date', 'weekday', 'covid',
       'BoroName', 'index', 'BoroCode', 'Shape_Leng', 'Shape_Area', 'datetime',
       'conditions', 'properties.name', 'weekend', 'week_x', 'month', 'week_y',
       'week'],
      dtype='object')


c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


alt.VConcatChart(...)

In [202]:
# make column with weekday number
accident_data=get_weather_data(data,fname = "weather.csv")
accident_data['weekday'] = accident_data['date'].dt.weekday

# make column with week number
accident_data['week'] = accident_data['date'].dt.week
# month column
accident_data['month'] = accident_data['date'].dt.month

# for each month get the minimum week number
min_week = accident_data.groupby(['month'])['week'].min().reset_index()
# merge with accident data
accident_data = pd.merge(accident_data,min_week,on='month',how='left')

C:\Users\pamar\AppData\Local\Temp\ipykernel_6348\3805221766.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  accident_data['week'] = accident_data['date'].dt.week


Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE',
       'LONGITUDE', 'VEHICLE TYPE CODE 1', 'date', 'weekday', 'covid',
       'BoroName', 'index', 'BoroCode', 'Shape_Leng', 'Shape_Area', 'datetime',
       'conditions', 'properties.name', 'week_x', 'month', 'week_y', 'week'],
      dtype='object')